In [1]:
import requests
from IPython.display import Image, display, clear_output
import time
from urllib.parse import urlencode
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO



In [2]:
# Function to set motors
def set_motors(jetbot_ip, left_speed, right_speed):
    params = {'left': left_speed, 'right': right_speed}
    url = f'http://{jetbot_ip}:8080/set_motors?{urlencode(params)}'
    response = requests.get(url)
    if response.status_code == 200:
        print("Motors set successfully")
    else:
        print("Failed to set motors")

In [3]:
# Function to execute left command
def move_left(jetbot_ip, speed):
    params = {'speed': speed}
    url = f'http://{jetbot_ip}:8080/left?{urlencode(params)}'
    response = requests.get(url)
    if response.status_code == 200:
        print("Left command executed successfully")
    else:
        print("Failed to execute left command")

In [4]:
# Function to execute right command
def move_right(jetbot_ip, speed):
    params = {'speed': speed}
    url = f'http://{jetbot_ip}:8080/right?{urlencode(params)}'
    response = requests.get(url)
    if response.status_code == 200:
        print("Right command executed successfully")
    else:
        print("Failed to execute right command")

In [5]:
# Function to execute forward command
def move_forward(jetbot_ip, speed):
    params = {'speed': speed}
    url = f'http://{jetbot_ip}:8080/forward?{urlencode(params)}'
    response = requests.get(url)
    if response.status_code == 200:
        print("Forward command executed successfully")
    else:
        print("Failed to execute forward command")

In [6]:
# Function to execute stop command
def stop_robot(jetbot_ip):
    url = f'http://{jetbot_ip}:8080/stop'
    response = requests.get(url)
    if response.status_code == 200:
        print("Stop command executed successfully")
    else:
        print("Failed to execute stop command")

In [7]:

# Function to display continuous camera stream
def display_camera_stream(jetbot_ip):
    while True:
        response = requests.get(f'http://{jetbot_ip}:8080/camera')
        image = Image(response.content)
        clear_output(wait=True)
        display(image)
        time.sleep(0.1) # Adjust the sleep time to control the frame rate
        
def print_camera(jetbot_ip):
    url = f'http://{jetbot_ip}:8080/camera'
    
    while True:
        response = requests.get(url)
        img_array = np.frombuffer(response.content, dtype=np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        # # Define HSV range for the color (red in this case)
        # lower_color = np.array([0, 120, 70])    # Lower bound of red
        # upper_color = np.array([10, 255, 255])  # Upper bound of red

        # HSV color range for pink
        lower_color = np.array([140, 50, 50])  # Lower bound of pink in HSV
        upper_color = np.array([170, 255, 255])  # Upper bound of pink in HSV


        if image is not None:

            # Convert the image to RGB
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Create mask for the color
            mask = cv2.inRange(rgb, lower_color, upper_color)

            # Find contours
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for cnt in contours:
                area = cv2.contourArea(cnt)
                if area > 500:  # Ignore small areas
                    x, y, w, h = cv2.boundingRect(cnt)
                    center_x = x + w // 2
                    center_y = y + h // 2

                    # Draw bounding box and center
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.circle(image, (center_x, center_y), 5, (0, 0, 255), -1)
                    cv2.putText(image, f"({center_x}, {center_y})", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

            cv2.imshow("JetBot Camera with Color Detection", image)


        
        # if image is not None:
        #     # Adjust color channels to reduce purpleness
        #     image[:, :, 0] = cv2.addWeighted(image[:, :, 0], 0.9, 0, 0, 0)  # Reduce blue channel
        #     image[:, :, 2] = cv2.addWeighted(image[:, :, 2], 0.9, 0, 0, 0)  # Reduce red channel
            
        #     cv2.imshow("JetBot Camera", image)
        
        # Break on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()



In [8]:

# A function that continuously streams video and allows saving images on pressing 's' and quitting on 'q'
def take_pictures_loop(jetbot_ip):
    folder = 'training_images'
    os.makedirs(folder, exist_ok=True)  # Create the folder if it doesn't exist

    url = f'http://{jetbot_ip}:8080/camera'

    while True:
        response = requests.get(url)
        img_array = np.frombuffer(response.content, dtype=np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        if image is not None:
            cv2.imshow("JetBot Camera - Press 's' to save, 'q' to quit", image)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('s'):  # Save image on pressing 's'
                # Count the number of files in the folder
                file_count = len([name for name in os.listdir(folder) if os.path.isfile(os.path.join(folder, name))])

                # Generate the new file name
                new_file_name = f'img{file_count + 1}.jpg'
                new_file_path = os.path.join(folder, new_file_name)

                cv2.imwrite(new_file_path, image)
                print(f"Image saved as {new_file_path}")
            elif key == ord('q'):  # Quit on pressing 'q'
                break
        else:
            print("Failed to capture image")

    cv2.destroyAllWindows()

In [9]:
# from ultralytics import YOLO

# # Load the YOLOv8 model
# model = YOLO("yolov8n.pt")  # You can replace "yolov8n.pt" with other YOLOv8 variants like "yolov8s.pt"

# # Train the model on the dataset
#     model.train(data="datasets/data.yaml", epochs=50, imgsz=640)

# # Save the trained model
# model.save("models/yolov8n_trained.pt")

In [ ]:



# Function to do object detection on all images in a folder and plot results
def detect_objects_in_folder(folder_path):
    images = []
    titles = []
    model = YOLO('yolov8n.pt')  # Load the YOLOv8 model

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            image_path = os.path.join(folder_path, filename)
            print(f"Processing {image_path}...")

            # Load the image
            image = cv2.imread(image_path)

            # Perform object detection
            results = model.predict(source=image, conf=0.25, save=False)

            # Annotate the image with detections
            for result in results:
                boxes = result.boxes.xyxy  # Bounding boxes
                confidences = result.boxes.conf  # Confidence scores
                class_ids = result.boxes.cls  # Class IDs

                for box, conf, class_id in zip(boxes, confidences, class_ids):
                    x1, y1, x2, y2 = map(int, box)
                    label = f"Class {int(class_id)}: {conf:.2f}"
                    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Convert BGR to RGB for matplotlib
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(image_rgb)
            titles.append(filename)

    # Plot the images with detections
    num_images = len(images)
    cols = 3
    rows = (num_images + cols - 1) // cols  # Calculate rows needed

    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()

    for i, ax in enumerate(axes):
        if i < num_images:
            ax.imshow(images[i])
            ax.set_title(titles[i])
            ax.axis('off')
        else:
            ax.axis('off')  # Hide unused subplots

    plt.tight_layout()
    plt.show()

In [11]:
def print_camera_yolo(jetbot_ip):
    # Load the trained YOLOv8 model
    model = YOLO("models/yolov8n_trained.pt")  # Load the trained model

    url = f'http://{jetbot_ip}:8080/camera'
    
    while True:
        response = requests.get(url)
        img_array = np.frombuffer(response.content, dtype=np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        if image is not None:
            # Run YOLO detection (pass BGR image directly)
            results = model.predict(image, verbose=False)

            for result in results:
                boxes = result.boxes
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = float(box.conf[0])
                    cls_id = int(box.cls[0])
                    label = model.names[cls_id]

                    # Draw bounding box and label
                    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(image, f'{label} {conf:.2f}', (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

                    # Draw center point
                    center_x = (x1 + x2) // 2
                    center_y = (y1 + y2) // 2
                    cv2.circle(image, (center_x, center_y), 5, (0, 0, 255), -1)
                    cv2.putText(image, f"({center_x},{center_y})", (x1, y2 + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Display the processed image
            cv2.imshow("JetBot Camera with YOLO Detection", image)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()


In [12]:
# Example usage
jetbot_ip = '194.47.156.140' # Replace with your Jetbot's IP address
# set_motors(jetbot_ip, 0.3, 0)
# move_left(jetbot_ip, 0.3)
# move_right(jetbot_ip, 0.3)
# move_forward(jetbot_ip, 0.3)
# stop_robot(jetbot_ip)
# display_camera_stream(jetbot_ip)
# print_camera(jetbot_ip)
# print_camera_yolo(jetbot_ip)
# take_pictures_loop(jetbot_ip)
detect_objects_in_folder("datasets/test/images")

Processing datasets/test/images/img14_jpg.rf.979fffd01db474e541221818f705eee5.jpg...


NameError: name 'model' is not defined